<a href="https://colab.research.google.com/github/sahilfatima/MLP_-_hypothesis_tests_on_Consumer_Marketing_data/blob/main/MLP_%26_hypothesis_tests_on_Consumer_Marketing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'marketing-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2452322%2F4153125%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240201%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240201T162515Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D12542489149225927a5a3d5554c67877eedc18f69251400c5d2ff02463b9507e5ef5c048f8dce28eb6cd4e8a36c5fec7720003b8a6c727e9669be21f92a76e0f126b431239498b08524e95f1313b821cc924a01cbd44fe57e0a03ede56691321d0659fb7294bd8dc9fc6678b9568d1cb9e112dacd7f0ae92a161fb9c6c64afc1aa1ba8aaab26b4c113ade1c6d9652808d3e08a3bc37a383cf39be3fadd110a38ced18fa7301e743d2f0ee09e38cda955f478fea8b263a4c10e2295e723d8b5f1cb10d286e9c7bdbdc5ea4b5d7c3c88391976b0f7226ba45d3a929ce73cc61e8178fc50b3b5bc51b7641694290be56035410e8c14ee37c5aac81f6fc39a6292a3'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

***The objective of the notebook is to implement statistical approaches and machine learning algorithms on a primary marketing data set. As a result, this approach goes on to explore the characteristics of this dataset and conduct different hypothesis tests on the dataset. The purpose of statistical hypothesis testing is to put the assumptions to the test and derive conclusions about
the population. This is accomplished by evaluating a sample representative of the entire population and, depending on the results, perhaps rejecting or accepting the hypothesis. We investigated assumptions about a population parameter using a range of hypothesis tests to support or reject hypotheses (statements).
Furthermore, the multiple linear regression model was developed to determine the association between various factors in a specified marketing dataset. Multiple linear regression is an improved variant of basic linear regression that is typically preferable for more complicated interactions that need additional considerations. To evaluate the created multiple linear
regression model accuracy, R-squared was utilized. It is a statistical parameter that evaluates how
close the data are to the fitted regression line.
All hypothesis tests and MLP models were done step-by-step in the Kaggle notebook IDE with the Python programming language. Python packages such as Pandas, Plotly, and Matplotlib were used during data processing and visualization. Whereas, SciPy and Scikit have been used to test hypotheses and develop the machine-learning model.***

In [ ]:
import pandas as pd
import plotly
import matplotlib
import scipy
import numpy as np
import sklearn

In [ ]:
data= pd.read_csv("/kaggle/input/marketing-data/marketing_data.csv", engine= 'python',encoding= "ISO-8859-1")

In [ ]:
data.head(5)

In [ ]:
data.info()

In [ ]:
data.rename(columns={' Income ': 'Earnings'}, inplace=True)
data['Earnings'].fillna(0, inplace=True)  # Replacing NaN with 0 or another appropriate value
data["Earnings"] = pd.to_numeric(data["Earnings"], errors='coerce')


In [ ]:
data['age']=2021-data['Year_Birth']
data['expenditure']=data['MntWines']+data['MntFruits']+data['MntMeatProducts']+data['MntFishProducts']+data['MntSweetProducts']+data['MntGoldProds']
data['marriage_status']=data['Marital_Status'].replace({'Divorced':'Single','Single':'Single','Married':'Couple','Together':'Couple','Absurd':'Single','Widow':'Single','YOLO':'Single', 'Alone':'Single'})
data['childerns'] = np.where(data.Kidhome+data.Teenhome > 0, 'Yes', 'No')

In [ ]:
data.head(5)

In [ ]:
data1= data[['age','expenditure','Earnings','marriage_status','childerns','Education']]

In [ ]:
data1.head(5)

# Statistical Summary

In [ ]:
pd.options.display.float_format = "{:.2f}".format
data1.describe()

# Handling Outliers

In [ ]:
import plotly.graph_objs as go
import plotly.express as px

In [ ]:
import plotly.express as px

df = px.data.tips()
fig = px.violin(data1, y="Earnings", color="marriage_status",
                violinmode='overlay',
                hover_data=data1.columns)
fig.show()

In [ ]:
df = px.data.tips()
fig = px.violin(data1, y="expenditure", x="childerns", color="marriage_status", box=True, points="all",
          hover_data=data1.columns)
fig.show()

In [ ]:
df = px.data.tips()
fig = px.strip(data1, x='Earnings', y='Education')
fig.show()

In [ ]:
df = px.data.tips()
fig = px.strip(data1, x='age', y='marriage_status')
fig.show()

In [ ]:
fig = go.Figure(data=go.Violin(y=data1['Earnings'], box_visible=True, line_color='black',
                               meanline_visible=True, fillcolor='lightseagreen', opacity=0.6,
                               x0='Total Earnings'))

fig.update_layout(yaxis_zeroline=False)
fig.show()

In [ ]:
data1=data1[data1['Earnings']<600000].reset_index(drop=True)

In [ ]:
fig = go.Figure(data=go.Violin(y=data1['Earnings'], box_visible=True, line_color='black',
                               meanline_visible=True, fillcolor='lightseagreen', opacity=0.6,
                               x0='Total Earnings'))

fig.update_layout(yaxis_zeroline=False)
fig.show()

In [ ]:
data1["Earnings"] = data1["Earnings"].astype(int)

# Checking Null Values

In [ ]:
data1.isnull().sum()

In [ ]:
data1.dropna(inplace=True)

In [ ]:
data1.info()

# EDA

In [ ]:
q=data1.groupby(['marriage_status']).size().reset_index(name='counts')
piechart=px.pie(q,values='counts', names='marriage_status', title='Marital Ratio')
piechart.show()

In [ ]:
q=data1.groupby(['Education']).size().reset_index(name='counts')
piechart=px.pie(q,values='counts', names='Education', title='Education Ratio')
piechart.show()

In [ ]:
data2=data1[['age','expenditure','Earnings','marriage_status','childerns','Education']]

In [ ]:
data1['Education']=data1['Education'].replace({'Basic':5,'2n Cycle':10,'Graduation':15,'Master':20,'PhD':25})

In [ ]:
data1['marriage_status']=data1['marriage_status'].replace({'Couple':2,'Single':1})

In [ ]:
data1['childerns']=data1['childerns'].replace({'Yes':2,'No':1})

In [ ]:
data1.head()

# Test For Normality (D'Agostino's K2 test)

In [ ]:
from scipy.stats import normaltest
a=data1[['Education']]
b=data1[['age']]
c=data1[['expenditure']]
d=data1[['marriage_status']]
e=data1[['childerns']]
f=data1[['Earnings']]
stat, p = normaltest(a)
print('Statistics=%.3f, p=%.3f' % (stat, p))
alpha = 0.05
if p > alpha:
    print('Eduacation: Sample looks Gaussian (fail to reject H0)')
else:
    print('Eduaction:  Sample does not look Gaussian (reject H0)')


stat, p = normaltest(b)
print('Statistics=%.3f, p=%.3f' % (stat, p))
alpha = 0.05
if p > alpha:
    print('age: Sample looks Gaussian (fail to reject H0)')
else:
    print('age: Sample does not look Gaussian (reject H0)')

stat, p = normaltest(c)
print('Statistics=%.3f, p=%.3f' % (stat, p))
alpha = 0.05
if p > alpha:
    print('expenditure: Sample looks Gaussian (fail to reject H0)')
else:
    print('expenditure: Sample does not look Gaussian (reject H0)')

stat, p = normaltest(d)
print('Statistics=%.3f, p=%.3f' % (stat, p))
alpha = 0.05
if p > alpha:
    print('marriage status: Sample looks Gaussian (fail to reject H0)')
else:
    print('marriage status: Sample does not look Gaussian (reject H0)')

stat, p = normaltest(e)
print('Statistics=%.3f, p=%.3f' % (stat, p))
alpha = 0.05
if p > alpha:
    print('childerns: Sample looks Gaussian (fail to reject H0)')
else:
    print('childerns: Sample does not look Gaussian (reject H0)')


stat, p = normaltest(f)
print('Statistics=%.3f, p=%.3f' % (stat, p))
alpha = 0.05
if p > alpha:
    print('earnings: Sample looks Gaussian (fail to reject H0)')
else:
    print('earnings: Sample does not look Gaussian (reject H0)')

In [ ]:
from scipy import stats
from numpy import median
from numpy import std

# Mann-Whitney's Test

In [ ]:
Diploma=data2[['Education','Earnings']]
Phd_degree=Diploma[Diploma['Education']=='PhD']
Master_degree=Diploma[Diploma['Education']=='Master']
Basic_degree=Diploma[Diploma['Education']=='Basic']
Second_degree=Diploma[Diploma['Education']=='2n Cycle']
Graduation_degree=Diploma[Diploma['Education']=='Graduation']

In [ ]:
Diploma.head(5)

In [ ]:
from scipy.stats import shapiro
from scipy.stats import normaltest

In [ ]:
print('PhD_degree median = %.0f stdv = %.1f' % (median(Phd_degree.Earnings), std(Phd_degree.Earnings)))
print('Master_degree median = %.0f stdv = %.1f' % (median(Master_degree.Earnings), std(Master_degree.Earnings)))

print(stats.mannwhitneyu(Phd_degree.Earnings, Master_degree.Earnings))

if p > alpha:
    print('We fail to reject H0 since the eanrings of PhD and master owners are not statistically different')
else:
    print('We reject H0 since the eanrings of PhD and master owners are statistically different')

# Spearman Correlation Test

In [ ]:
expenditure=data1[['expenditure','Earnings']]
cor, pval = stats.spearmanr(expenditure[['expenditure']], expenditure[['Earnings']])
print("The orrelation coefficient of Non-Parametric Spearman correlation test : %.4f, \np-value: %.4f" % (cor, pval))

# Chi-Sqaured Test

In [ ]:
from scipy.stats import chi2_contingency
from scipy.stats import chi2

In [ ]:
Marital_Situation=data2[['Education','marriage_status']]
crosstab = pd.crosstab(Marital_Situation["Education"], Marital_Situation["marriage_status"])
crosstab

# Degree Of Freedom

In [ ]:
stat, p, dof, expected = chi2_contingency(crosstab)
print('Degree of freedom for chi squared test = %d' % dof)

# Expected Frequancy

In [ ]:
print(expected)

# P-Value

In [ ]:
prob = 0.95
critical = chi2.ppf(prob, dof)
print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))

# Independence

In [ ]:
if abs(stat) >= critical:
	print('We reject H0, the variables are dependent')
else:
	print('We fail to reject H0, the variables are independent')
alpha = 1.0 - prob
print('significance=%.3f, p=%.3f' % (alpha, p))
if p <= alpha:
	print('We reject H0, the variables are dependent')
else:
	print('We fail to reject H0, the variables are independent')

# Multiple Linear Regression

In [ ]:
data2.info()

# Checking Correlations

In [ ]:
data2.corr()

In [ ]:
X=data2[['age','expenditure','Education']]

**Target variable is earnings while age, expenditure and education are independent variables.**

In [ ]:
X.head()

In [ ]:
y=data2['Earnings']

In [ ]:
y.head()

In [ ]:
X.reset_index()
y.reset_index()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score


# One_Hot_Encoder

In [ ]:
ohe=OneHotEncoder(handle_unknown= "ignore")

ohe.fit_transform(X[["Education"]])

In [ ]:
ohe.categories_

In [ ]:
column_trans=make_column_transformer((OneHotEncoder(categories=ohe.categories_),["Education"]),
                                    remainder='passthrough')

In [ ]:
column_trans.fit_transform(X)

In [ ]:
lr=LinearRegression()

In [ ]:
pipe=make_pipeline(column_trans,lr)

In [ ]:
pipe.fit(X_train,y_train)

In [ ]:
y_pred= pipe.predict(X_test)

# R-squared scores

In [ ]:
r2_score(y_test,y_pred)

In [ ]:
scores=[]
for i in range(1000):
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=i)
    lr=LinearRegression()
    pipe=make_pipeline(column_trans,lr)
    pipe.fit(X_train,y_train)
    y_pred=pipe.predict(X_test)
    scores.append(r2_score(y_test,y_pred))

In [ ]:
np.argmax(scores)

In [ ]:
scores[np.argmax(scores)]

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=np.argmax(scores))
lr=LinearRegression()
pipe=make_pipeline(column_trans,lr)
pipe.fit(X_train,y_train)
y_pred=pipe.predict(X_test)
r2_score(y_test,y_pred)

# Scattar Plot

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_test, y_pred)
ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')

y_test, y_pred = y_test.values.reshape(-1,1), y_pred.reshape(-1,1)
ax.plot(y_test, LinearRegression().fit(y_test, y_pred).predict(y_test))
resolution_value = 300
plt.savefig("myImage.pdf", format="pdf", dpi=resolution_value)
plt.show()
